### A lot of duplicated code with no reuse whatsoever. Not good but way faster...

In [1]:
import os
import torch
import pprint
import pandas as pd
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from PIL import Image
from torchvision import transforms as T
import torchvision
from tqdm import tqdm

In [2]:
import sys

sys.path.append("../")

from src.detection.prediction_utils import get_model, filter_predictions, visualize_boxes
from src.slides_utils.slides_utils import *
from src.line_detection.hough import get_tablas

## Instance models

In [3]:
PATH_MODELS = "/home/nacho/TFI-Cazcarra/models"

test_df = pd.read_csv("/home/nacho/TFI-Cazcarra/data/csv/test_diagramas_2023.csv")

In [5]:
retinanet = get_model(object_to_predict="tablas")

In [ ]:
faster_rcnn = choose_model(model_name="fasterrcnn", object_to_predict="tablas")

In [ ]:
yolo = torch.hub.load('ultralytics/yolov5', 'custom', os.path.join(PATH_YOLO, "exp7", "weights", "best_tablas.pt"))

## Calculo las predicciones de las tablas

In [6]:
where_to_search = ["/home/nacho/TFI-Cazcarra/data/csv/test_diagramas_2023.csv"]
transform = T.Compose([T.ToTensor()])

In [7]:
def gen_target(tablas):
    return {"boxes": torch.Tensor(tablas), "labels": torch.IntTensor([1]*len(tablas))}

def gen_pred(tablas_boxes, tablas_scores):
    return {"boxes": tablas_boxes, "scores": tablas_scores, "labels": torch.IntTensor([1]*len(tablas_boxes))}

In [11]:
def calculate_metrics_torch(model, test_df):
    model.eval()
    preds = []
    target = []

    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img)
        tablas, _ = get_tablas(img_path, where_to_search)
        with torch.no_grad():
            tablas_pred = model([img_tensor])[1][0]
        tablas_boxes, tablas_scores = filter_predictions(tablas_pred, nms_threshold=0.5, score_threshold=0.49)
        preds.append(gen_pred(tablas_boxes, tablas_scores))
        target.append(gen_target(tablas))

    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [9]:
def calculate_metrics_yolo(model, test_df):
    preds = []
    target = []

    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        tablas, _ = get_tablas(img_path, where_to_search)
        tablas_pred = model(img)

        pred = {"boxes": tablas_pred.xyxy[0][:, :4], "scores": tablas_pred.xyxy[0][:, 4]}
        tablas_boxes, tablas_scores = filter_predictions(pred, nms_threshold=0.5, score_threshold=0.49)
        
        preds.append(gen_pred(tablas_boxes, tablas_scores))
        target.append(gen_target(tablas))
        
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [ ]:
calculate_metrics_torch(faster_rcnn, test_df)

In [ ]:
calculate_metrics_yolo(yolo, test_df)

## Calculo las predicciones para las cardinalidades
NOTA: Como las labels no importan, las seteamos todas a 1. Lo único que necesita el sistema es saber donde están las cardinalidades, no de qué tipo son.

In [ ]:
retinanet = torch.jit.load(PATH_MODELS+"/"+"model_best_cardinalidades_retinanet.pt")

In [ ]:
yolo = torch.hub.load('ultralytics/yolov5', 'custom', os.path.join(PATH_YOLO, "exp9", "weights", "best_cardinalidades_retrained.pt"))

In [ ]:
where_to_search = ["/home/nacho/TFI-Cazcarra/data/tiles/test_cardinalidades_2023_fixed.csv"]
transform = T.Compose([T.ToTensor()])

In [ ]:
def gen_target_card(boxes_gt, label_gt):
    return {"boxes": boxes_gt, "labels": label_gt}

def gen_pred_card(boxes, scores, labels):
    return {"boxes": boxes, "scores": scores, "labels": labels}

def filter_predictions2(predictions, score_threshold=0.5, nms_threshold=0.5):
    boxes = predictions['boxes'][predictions['scores'] >= score_threshold]
    scores = predictions['scores'][predictions['scores'] >= score_threshold]
    labels = predictions['labels'][predictions['scores'] >= score_threshold]
    valid_idx = torchvision.ops.nms(boxes, scores, nms_threshold)
    return boxes[valid_idx], scores[valid_idx], labels[valid_idx]

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [ ]:
def calculate_metrics_torch(model, test_df):
    model.eval()
    preds = []
    target = []

    le_dict = {'muchos_opcional': 2,
               'muchos_obligatorio': 1,
               'uno_opcional': 3,
               'uno_obligatorio': 4}
    
    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img)
        
        boxes_gt = test_df[test_df.image_path==img_path][['xmin','ymin','xmax','ymax']].values
        boxes_gt = torch.Tensor(boxes_gt)
        label_gt = test_df[test_df.image_path==img_path]['label'].values
        label_gt = torch.Tensor([le_dict[val] for val in label_gt])
        
        with torch.no_grad():
            card_pred = model([img_tensor])[1][0]
        card_boxes, card_scores, card_labels = filter_predictions2(card_pred, 
                                                                  nms_threshold=0.5, 
                                                                 score_threshold=0.3)
        img = visualize_boxes(img, card_boxes, color=(255,0,0))
        img = visualize_boxes(img, boxes_gt, color=(0,255,0))
        display(img_path, img)
        preds.append(gen_pred(card_boxes, card_scores))
        target.append(gen_target(boxes_gt))

    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [ ]:
def calculate_metrics_yolo(model, test_df):
    preds = []
    target = []

    le_dict = {'muchos_opcional': 2,
           'muchos_obligatorio': 1,
           'uno_opcional': 3,
           'uno_obligatorio': 4}
    
    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        card_pred = model(img)
        w,h = img.size
        boxes_gt = test_df[test_df.image_path==img_path][['xmin','ymin','xmax','ymax']].values
        boxes_gt = torch.Tensor(boxes_gt)
        label_gt = test_df[test_df.image_path==img_path]['label'].values
        label_gt = torch.Tensor([le_dict[val]-1 for val in label_gt])
        
        pred = {"boxes": card_pred.xyxy[0][:, :4], "scores": card_pred.xyxy[0][:, 4], 
                "labels": card_pred.xyxy[0][:, 5]}

        card_boxes, card_scores, card_labels = filter_predictions2(pred, nms_threshold=0.5, score_threshold=0.49)
        
        img = visualize_boxes(img, card_boxes, color=(255,0,0))
        img = visualize_boxes(img, boxes_gt, color=(0,255,0))
        display(img_path, img)
        #Cambio esto y no le paso las labels a ver que pasa...
        preds.append(gen_pred(card_boxes, card_scores))
        target.append(gen_target(boxes_gt))
        
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [ ]:
test_df = pd.read_csv("/home/nacho/TFI-Cazcarra/data/tiles/test_cardinalidades_2023_fixed.csv")

In [ ]:
calculate_metrics_torch(retinanet, test_df)

In [ ]:
calculate_metrics_yolo(yolo, test_df)

## Lo mismo para tiles PERO calculo los scores para las imagenes enteras

In [ ]:
test_df = pd.read_csv("/home/nacho/TFI-Cazcarra/data/csv/test_diagramas_2023.csv")
where_to_search = ["/home/nacho/TFI-Cazcarra/data/csv/test_diagramas_2023.csv"]
transform = T.Compose([T.ToTensor()])

In [ ]:
def unify_images2(img, boxes_per_tile):
    img = np.array(img)
    first_tile = next(iter(boxes_per_tile.keys()))
    all_boxes = np.array([[]])
    all_scores = np.array([])
    all_labels = np.array([])
    
    for tile, prediction in boxes_per_tile.items():
        coords_to_add = torch.Tensor(list(map(lambda i,j: i-j, literal_eval(tile), literal_eval(first_tile))))
        boxes = torch.add(prediction['boxes'], coords_to_add, alpha=1).detach().numpy()
        all_boxes = np.append(all_boxes, boxes)
        all_scores = np.append(all_scores, prediction['scores'])
        all_labels = np.append(all_labels, prediction['labels'])
    return {"boxes": torch.from_numpy(all_boxes.reshape((-1,4))), "scores": torch.from_numpy(all_scores),
            "labels": torch.from_numpy(all_labels)}


def predict_tiles2(img, model, is_yolo, transform, min_size=600, max_size=1333):
    tiles = slice_img_from_prediction(img, tile_size=None, tile_overlap=0.0, number_tiles=6)
    preds_image = {}
    with torch.no_grad():
        for tile in tiles:
            tile_img = img.crop(tile).convert("RGB")
            tensor_tile = transform(tile_img)
            if not is_yolo:
                predictions = model([tensor_tile])[1][0]
            else:
                predictions = model(tile_img)
                predictions = {"boxes": predictions.xyxy[0][:, :4], 
                               "scores": predictions.xyxy[0][:, 4],
                               "labels": predictions.xyxy[0][:, 5]}
            preds_image[str(tile)] = predictions
    unified_results = unify_images2(img=img, boxes_per_tile=preds_image)
    return unified_results

In [ ]:
def calculate_metrics_torch(model, test_df):
    model.eval()
    preds = []
    target = []
    test_df = test_df[test_df['label']!="tabla"]
    le_dict = {'muchos_opcional': 2,
               'muchos_obligatorio': 1,
               'uno_opcional': 3,
               'uno_obligatorio': 4}
    
    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img)
        
        boxes_gt = test_df[test_df.image_path==img_path][['xmin','ymin','xmax','ymax']].values
        boxes_gt = torch.Tensor(boxes_gt)
        label_gt = test_df[test_df.image_path==img_path]['label'].values
        label_gt = torch.Tensor([le_dict[val] for val in label_gt])
        
        with torch.no_grad():
            cardinalidades_pred = predict_tiles2(img, model=model, is_yolo=False, transform=transform)
            card_boxes, card_scores, card_labels = filter_predictions2(cardinalidades_pred, nms_threshold=0.5, 
                                                                       score_threshold=0.3)       
        
        preds.append(gen_pred(card_boxes, card_scores))
        target.append(gen_target(boxes_gt))

        img = visualize_boxes(img, card_boxes, color=(255,0,0))
        img = visualize_boxes(img, boxes_gt, color=(0,255,0))
        display(img_path, img)
        
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [ ]:
def calculate_metrics_yolo_unified(model, test_df):
    
    test_df = test_df[test_df['label']!="tabla"]
    preds = []
    target = []
    
    le_dict = {'muchos_opcional': 2,
               'muchos_obligatorio': 1,
               'uno_opcional': 3,
               'uno_obligatorio': 4}
        
    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        
        cardinalidades_pred = predict_tiles2(img, model=model, is_yolo=True, transform=transform)
        cardinalidades_boxes, cardinalidades_scores, cardinalidades_labels = filter_predictions2(cardinalidades_pred, 
                                                                                                 nms_threshold=0.5, 
                                                                                                score_threshold=0.49)       
        boxes_gt = test_df[test_df.image_path==img_path][['xmin','ymin','xmax','ymax']].values
        boxes_gt = torch.Tensor(boxes_gt)
        label_gt = test_df[test_df.image_path==img_path]['label'].values
        label_gt = torch.Tensor([le_dict[val]-1 for val in label_gt])
    
        img = visualize_boxes(img, cardinalidades_boxes, color=(255,0,0))
        img = visualize_boxes(img, boxes_gt, color=(0,255,0))
        #display(img_path, img)
        
        preds.append(gen_pred(cardinalidades_boxes, cardinalidades_scores))
        target.append(gen_target(boxes_gt))
        
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [ ]:
calculate_metrics_yolo_unified(yolo, test_df)

In [ ]:
calculate_metrics_torch(faster_rcnn, test_df)